In [1]:
import json
import torch
import torch.distributed as dist
import torch.nn as nn
# import torch.optim as optim
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW
from tqdm import tqdm  # for our progress bar
from torch.nn.parallel import DistributedDataParallel as DDP

/media/trevor/SecondaryM2/VirginiaTech/HokieGuideliner/hokieguideliner/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
blen_tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")
blen_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-400M-distill")

inst_tokenizer = AutoTokenizer.from_pretrained("prakharz/DIAL-BART0")
inst_model = AutoModelForSeq2SeqLM.from_pretrained("prakharz/DIAL-BART0")

In [14]:
device_id = '0' # need to change this to 6 when I am training w/ jingyuan's GPU
max_length = 512
batch_size = 32
epoch_num = 50

device = torch.device('cuda:'+device_id)

In [26]:
UTTERANCE = "My friends are cool but they eat too many carbs. </s> <s>That\'s unfortunate. Are they trying to lose weight or are they just trying to be healthier?</s> <s>I'm not sure."
LABEL = "Well, maybe they are struggling?"

inputs = blen_tokenizer([UTTERANCE], return_tensors='pt').input_ids
lab = blen_tokenizer([LABEL], return_tensors='pt').input_ids

reply_ids = blen_model(input_ids=inputs, labels=lab)
print(reply_ids.loss)

tensor(3.1299, grad_fn=<NllLossBackward0>)


In [28]:
UTTERANCE = "My friends are cool but they eat too many carbs. </s> <s>That\'s unfortunate. Are they trying to lose weight or are they just trying to be healthier?</s> <s>I'm not sure."
inputs = blen_tokenizer([UTTERANCE], return_tensors='pt')
reply_ids = blen_model.generate(**inputs, max_new_tokens=50)
print(blen_tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0])

 I hope they can find a way to do it. I know it can be hard.


In [2]:
def get_topic_guideline(topics):
    history = ''
    if topics is None:
        return history
    if 'high-level' in topics and topics['high-level'] is not None and 'topic' in topics['high-level']:
        if type(topics['high-level']['topic']) is list:
            topic = ', '.join(topics['high-level']['topic'])
        else: 
            topic = topics['high-level']['topic']
        if topic is not None:
            history += 'These two people are talking about ' + topic + '.'
            if 'if_interest' in topics['high-level'] and topics['high-level']['if_interest'] == 'no':
                history += ' However, user B does not like this topic.'
            
    if 'middle-level' in topics and topics['middle-level'] is not None and 'topic' in topics['middle-level']:
        if type(topics['middle-level']['topic']) is list:
            topic = ', '.join(topics['middle-level']['topic'])
        else: 
            topic = topics['middle-level']['topic']
        if topic is not None:
            history += ' Specifically, they focus on ' + topic + '.'
            if 'if_interest' in topics['middle-level'] and topics['middle-level']['if_interest'] == 'no':
                history += ' However, user B does not like this.'
            
    if 'low-level' in topics and topics['low-level'] is not None and 'topic' in topics['low-level']:
        if type(topics['low-level']['topic']) is list:
            topic = ', '.join(topics['low-level']['topic'])
        else: 
            topic = topics['low-level']['topic']
        if topic is not None:
            history += ' In more detail, they are discussing about ' + topic + '.'
            if 'if_interest' in topics['low-level'] and topics['low-level']['if_interest'] == 'no':
                history += ' However, user B does not want to talk about it.'
            
    return history

In [3]:
data = json.load(open('./sample_conv4.json', 'r'))
train_data = data[:int(0.8*len(data))]
train_input = []
train_output = []
sep = ' [ENDOFTURN] '
instruction = 'Instruction: Generate a response that following the given topic guidline. \n\nInput: [TOPICS] '
cont = ' [CONTEXT] '
end = ' [ENDOFDIALOGUE] [QUESTION] Given this conversation provided, a response following the topic guideline is'
for conversation in train_data:
    dialogue_history = ""
    for idx, message in enumerate(conversation):
        if len(dialogue_history) != 0:
            if 'topic_dict' not in message:
                dialogue_history += sep
            else:
                dialogue_history += ' '
                topic = get_topic_guideline(message['topic_dict'])
        dialogue_history += message['message']
        dialogue_history = dialogue_history.replace('A: ', 'Robot: ')
        dialogue_history = dialogue_history.replace('B: ', 'Human: ')
        
        if 'topic_dict' in message:
            input_text = instruction + topic + cont + dialogue_history + end
            if message['topic_dict'] is not None and idx < len(conversation) - 1:
                output = conversation[idx + 1]['message']
                output = output.replace('A: ', 'Robot: ')
                output = output.replace('B: ', 'Human: ')
            
                
                train_input.append(input_text)
                train_output.append(output)

In [6]:
text_input = [line.strip() for line in train_input]
text_label = [line.strip() for line in train_output]
print(len(text_input), len(text_label))

12344 12344


In [12]:
for i in range(5):
    print(text_input[i])
    print(text_label[i])
    print('---')


Instruction: Generate a response that following the given topic guidline. 

Input: [TOPICS] These two people are talking about emotions. Specifically, they focus on well-being. [CONTEXT] Robot: Hi, how are you doing today? Human: I'm doing pretty well. [ENDOFDIALOGUE] [QUESTION] Given this conversation provided, a response following the topic guideline is
Robot: That's good to hear. Have you heard the news about the upcoming election?
---
Instruction: Generate a response that following the given topic guidline. 

Input: [TOPICS] These two people are talking about news. However, user B does not like this topic. Specifically, they focus on politics. However, user B does not like this. [CONTEXT] Robot: Hi, how are you doing today? Human: I'm doing pretty well. [ENDOFTURN] Robot: That's good to hear. Have you heard the news about the upcoming election? Human: I have, but to be honest, I'm not too interested in politics. [ENDOFDIALOGUE] [QUESTION] Given this conversation provided, a respons

In [5]:
print("hello")

hello
